# H2O GBM Tuning Tutorial for Python

### Arno Candel, PhD, Chief Architect, H2O.ai
### Ported to Python by Navdeep Gill, M.S., Hacker/Data Scientist, H2O.ai

In this tutorial, we show how to build a well-tuned H2O GBM model for a supervised classification task. We specifically don't focus on feature engineering and use a small dataset to allow you to reproduce these results in a few minutes on a laptop. This script can be directly transferred to datasets that are hundreds of GBs large and H2O clusters with dozens of compute nodes.

You can download the source [from H2O's github repository](https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.ipynb).

Original [R Markdown version here](https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.Rmd).

## Installation of the H2O Python Package
Either download H2O from [H2O.ai's website](http://h2o.ai/download) or install the latest version of H2O into Python with the following set of commands:

Install dependencies from command line (prepending with `sudo` if needed):

```
[sudo] pip install -U requests
[sudo] pip install -U tabulate
[sudo] pip install -U future
[sudo] pip install -U six
```

The following command removes the H2O module for Python.
```
[sudo] pip uninstall h2o
```

Next, use pip to install this version of the H2O Python module.
```
[sudo] pip install http://h2o-release.s3.amazonaws.com/h2o/rel-turchin/7/Python/h2o-3.8.2.8-py2.py3-none-any.whl
```

## Launch an H2O cluster on localhost

In [ ]:
import h2o
import numpy as np
import math
h2o.init(nthreads=-1, strict_version_check=True)
## optional: connect to a running H2O cluster
#h2o.init(ip="mycluster", port=55555)

## Import the data into H2O 
Everything is scalable and distributed from now on. All processing is done on the fully multi-threaded and distributed H2O Java-based backend and can be scaled to large datasets on large compute clusters.
Here, we use a small public dataset ([Titanic](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/Titanic.html)), but you can use datasets that are hundreds of GBs large.

In [ ]:
## 'path' can point to a local file, hdfs, s3, nfs, Hive, directories, etc.
df = h2o.import_file(path = "http://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv")
print df.dim
print df.head
print df.tail
print df.describe

## pick a response for the supervised problem
response = "survived"

## the response variable is an integer, we will turn it into a categorical/factor for binary classification
df[response] = df[response].asfactor()           

## use all other columns (except for the name & the response column ("survived")) as predictors
predictors = df.columns
del predictors[1:3]
print predictors

From now on, everything is generic and directly applies to most datasets. We assume that all feature engineering is done at this stage and focus on model tuning. For multi-class problems, you can use `h2o.logloss()` or `h2o.confusion_matrix()` instead of `h2o.auc()` and for regression problems, you can use `h2o.mean_residual_deviance()` or `h2o.mse()`.

## Split the data for Machine Learning
We split the data into three pieces: 60% for training, 20% for validation, 20% for final testing. 
Here, we use random splitting, but this assumes i.i.d. data. If this is not the case (e.g., when events span across multiple rows or data has a time structure), you'll have to sample your data non-randomly.

In [ ]:
train, valid, test = df.split_frame(ratios=[0.6,0.2], seed=1234, destination_frames=['train.hex','valid.hex','test.hex'])

## Establish baseline performance
As the first step, we'll build some default models to see what accuracy we can expect. Let's use the [AUC metric](http://mlwiki.org/index.php/ROC_Analysis) for this demo, but you can use `h2o.logloss()` and `stopping_metric="logloss"` as well. It ranges from 0.5 for random models to 1 for perfect models.


The first model is a default GBM, trained on the 60% training split

In [ ]:
#We only provide the required parameters, everything else is default
gbm = h2o.H2OGradientBoostingEstimator(distribution='bernoulli')
gbm.train(x=predictors, y=response, training_frame=train)

## Show a detailed model summary
print gbm

The AUC is over 94%, so this model is highly predictive!

In [ ]:
## Get the AUC on the validation set
perf = gbm.model_performance(valid)
print perf.auc()

The second model is another default GBM, but trained on 80% of the data (here, we combine the training and validation splits to get more training data), and cross-validated using 4 folds.
Note that cross-validation takes longer and is not usually done for really large datasets.

In [ ]:
## rbind() makes a copy here, so it's better to use split_frame with `ratios = c(0.8)` instead above
cv_gbm = h2o.H2OGradientBoostingEstimator(distribution='bernoulli',nfolds = 4, seed = 0xDECAF)
cv_gbm.train(x = predictors, y = response, training_frame = train.rbind(valid))

We see that the cross-validated performance is similar to the validation set performance:

In [ ]:
## Show a detailed summary of the cross validation metrics
## This gives you an idea of the variance between the folds
cv_summary = cv_gbm.cross_validation_metrics_summary().as_data_frame()
#print(cv_summary) ## Full summary of all metrics
#print(cv_summary.iloc[4]) ## get the row with just the AUCs

## Get the cross-validated AUC by scoring the combined holdout predictions.
## (Instead of taking the average of the metrics across the folds)
perf_cv = cv_gbm.model_performance(xval=True)
print perf_cv.auc()

Next, we train a GBM with "I feel lucky" parameters.
We'll use early stopping to automatically tune the number of trees using the validation AUC. 
We'll use a lower learning rate (lower is always better, just takes more trees to converge).
We'll also use stochastic sampling of rows and columns to (hopefully) improve generalization.

In [ ]:
gbm_lucky = h2o.H2OGradientBoostingEstimator(
  ## more trees is better if the learning rate is small enough 
  ## here, use "more than enough" trees - we have early stopping
  ntrees = 10000,                                                            

  ## smaller learning rate is better (this is a good value for most datasets, but see below for annealing)
  learn_rate = 0.01,                                                         

  ## early stopping once the validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
  stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "AUC", 

  ## sample 80% of rows per tree
  sample_rate = 0.8,                                                       

  ## sample 80% of columns per split
  col_sample_rate = 0.8,                                                   

  ## fix a random number generator seed for reproducibility
  seed = 1234,                                                             

  ## score every 10 trees to make early stopping reproducible (it depends on the scoring interval)
  score_tree_interval = 10)

gbm_lucky.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

This model doesn't seem to be much better than the previous models:

In [ ]:
perf_lucky = gbm_lucky.model_performance(valid)
print perf_lucky.auc()

For this small dataset, dropping 20% of observations per tree seems too aggressive in terms of adding regularization. For larger datasets, this is usually not a bad idea. But we'll let this parameter tune freshly below, so no worries.

## Hyper-Parameter Search

Next, we'll do real hyper-parameter optimization to see if we can beat the best AUC so far (around 94%).

The key here is to start tuning some key parameters first (i.e., those that we expect to have the biggest impact on the results). From experience with gradient boosted trees across many datasets, we can state the following "rules":

1. Build as many trees (`ntrees`) as it takes until the validation set error starts increasing.
2. A lower learning rate (`learn_rate`) is generally better, but will require more trees. Using `learn_rate=0.02 `and `learn_rate_annealing=0.995` (reduction of learning rate with each additional tree) can help speed up convergence without sacrificing accuracy too much, and is great to hyper-parameter searches. For faster scans, use values of 0.05 and 0.99 instead.
3. The optimum maximum allowed depth for the trees (`max_depth`) is data dependent, deeper trees take longer to train, especially at depths greater than 10.
4. Row and column sampling (`sample_rate` and `col_sample_rate`) can improve generalization and lead to lower validation and test set errors. Good general values for large datasets are around 0.7 to 0.8 (sampling 70-80 percent of the data) for both parameters. Column sampling per tree (`col_sample_rate_per_tree`) can also be tuned. Note that it is multiplicative with `col_sample_rate`, so setting both parameters to 0.8 results in 64% of columns being considered at any given node to split.
5. For highly imbalanced classification datasets (e.g., fewer buyers than non-buyers), stratified row sampling based on response class membership can help improve predictive accuracy.  It is configured with `sample_rate_per_class` (array of ratios, one per response class in lexicographic order).
6. Most other options only have a small impact on the model performance, but are worth tuning with a Random hyper-parameter search nonetheless, if highest performance is critical.

First we want to know what value of `max_depth` to use because it has a big impact on the model training time and optimal values depend strongly on the dataset.
We'll do a quick Cartesian grid search to get a rough idea of good candidate `max_depth` values. Each model in the grid search will use early stopping to tune the number of trees using the validation set AUC, as before.
We'll use learning rate annealing to speed up convergence without sacrificing too much accuracy.

In [ ]:
## Depth 10 is usually plenty of depth for most datasets, but you never know
hyper_params = {'max_depth' : range(1,30,2)}
#hyper_params = {max_depth = c(4,6,8,12,16,20)} ##faster for larger datasets

#Build initial GBM Model
gbm_grid = h2o.H2OGradientBoostingEstimator(distribution='bernoulli',
        ## more trees is better if the learning rate is small enough 
        ## here, use "more than enough" trees - we have early stopping
        ntrees=10000,
        ## smaller learning rate is better
        ## since we have learning_rate_annealing, we can afford to start with a 
        #bigger learning rate
        learn_rate=0.05,
        ## learning rate annealing: learning_rate shrinks by 1% after every tree 
        ## (use 1.00 to disable, but then lower the learning_rate)
        learn_rate_annealing = 0.99,
        ## sample 80% of rows per tree
        sample_rate = 0.8,
        ## sample 80% of columns per split
        col_sample_rate = 0.8,
        ## fix a random number generator seed for reproducibility
        seed = 1234,
        ## score every 10 trees to make early stopping reproducible 
        #(it depends on the scoring interval)
        score_tree_interval = 10, 
        ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
        #5 consecutive scoring events
        stopping_rounds = 5,
        stopping_metric = "AUC",
        stopping_tolerance = 1e-4)

#Build grid search with previously made GBM and hyper parameters
grid = h2o.H2OGridSearch(gbm_grid,hyper_params,
                         grid_id = 'depth_grid',
                         search_criteria = {'strategy': "Cartesian"})


#Train grid search
grid.train(x=predictors, 
           y=response,
           training_frame = train,
           validation_frame = valid)

In [ ]:
## by default, display the grid search results sorted by increasing logloss (since this is a classification task)
print grid

In [ ]:
## sort the grid models by decreasing AUC
sorted_grid = grid.sort_by('auc(valid=True)',increasing=False)
print(sorted_grid)

In [ ]:
# find the range of the max_depth for the top ten models
top_depths = sorted_grid[0:5]
max_depths = top_depths['Hyperparameters: [max_depth]']
print max_depths

# get the max depths as a list
max_min_list = []
for element in max_depths:
    max_min_list.append(element[0])

It appears that `max_depth` values of 19 to 29 are best suited for this dataset, which is unusally deep!

In [ ]:
new_max = max(max_min_list)
new_min = min(max_min_list)

print "MaxDepth", new_max
print "MinDepth", new_min

Now that we know a good range for max_depth, we can tune all other parameters in more detail. Since we don't know what combinations of hyper-parameters will result in the best model, we'll use random hyper-parameter search to "let the machine get luckier than a best guess of any human".

In [ ]:
# create hyperameter and search criteria lists (ranges are inclusive..exclusive))
hyper_params_tune = {'max_depth' : list(range(new_min,new_max+1,1)),
                'sample_rate': [x/100. for x in range(20,101)],
                'col_sample_rate' : [x/100. for x in range(20,101)],
                'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
                'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                'min_rows': [2**x for x in range(0,int(math.log(train.nrow,2)-1)+1)],
                'nbins': [2**x for x in range(4,11)],
                'nbins_cats': [2**x for x in range(4,13)],
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["UniformAdaptive","QuantilesGlobal","RoundRobin"]}
search_criteria_tune = {'strategy': "RandomDiscrete",
                   'max_runtime_secs': 3600,  ## limit the runtime to 60 minutes
                   'max_models': 100,  ## build no more than 100 models
                   'seed' : 1234,
                   'stopping_rounds' : 5,
                   'stopping_metric' : "AUC",
                   'stopping_tolerance': 1e-3
                   }

In [ ]:
gbm_final_grid = h2o.H2OGradientBoostingEstimator(distribution='bernoulli',
                    ## more trees is better if the learning rate is small enough 
                    ## here, use "more than enough" trees - we have early stopping
                    ntrees=10000,
                    ## smaller learning rate is better
                    ## since we have learning_rate_annealing, we can afford to start with a 
                    #bigger learning rate
                    learn_rate=0.05,
                    ## learning rate annealing: learning_rate shrinks by 1% after every tree 
                    ## (use 1.00 to disable, but then lower the learning_rate)
                    learn_rate_annealing = 0.99,
                    ## score every 10 trees to make early stopping reproducible 
                    #(it depends on the scoring interval)
                    score_tree_interval = 10,
                    ## fix a random number generator seed for reproducibility
                    seed = 1234,
                    ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
                    #5 consecutive scoring events
                    stopping_rounds = 5,
                    stopping_metric = "AUC",
                    stopping_tolerance = 1e-4)
            
#Build grid search with previously made GBM and hyper parameters
final_grid = h2o.H2OGridSearch(gbm_final_grid, hyper_params = hyper_params_tune,
                                    grid_id = 'final_grid',
                                    search_criteria = search_criteria_tune)
#Train grid search
final_grid.train(x=predictors, 
           y=response,
           ## early stopping based on timeout (no model should take more than 1 hour - modify as needed)
           max_runtime_secs = 3600, 
           training_frame = train,
           validation_frame = valid)

print final_grid

We can see that the best models have even better validation AUCs than our previous best models, so the random grid search was successful!

In [ ]:
## Sort the grid models by AUC
sorted_final_grid = final_grid.sort_by('auc(valid=True)',increasing=False)
print sorted_final_grid

We can inspect the best 5 models from the grid search explicitly, and query their validation AUC:

In [ ]:
print sorted_final_grid["auc(valid=True)"][0:5]

You can also see the results of the grid search in [Flow](http://localhost:54321/):
![alt text](./final_grid.png "Final Grid Search Results in Flow")

## Model Inspection and Final Test Set Scoring

Let's see how well the best model of the grid search (as judged by validation set AUC) does on the held out test set:

In [ ]:
#Get the best model from the list (the model name listed at the top of the table)
best_model = h2o.get_model(sorted_final_grid['Model Id'][0])
performance_best_model = best_model.model_performance(test)

Good news. It does as well on the test set as on the validation set, so it looks like our best GBM model generalizes well to the unseen test set:

In [ ]:
#Get the performance on the test model
print performance_best_model.auc()

We can inspect the winning model's parameters:

In [ ]:
params_list = []
for key, value in best_model.params.iteritems():
    params_list.append(str(key)+" = "+str(value['actual']))
params_list

Now we can confirm that these parameters are generally sound, by building a GBM model on the whole dataset (instead of the 60%) and using internal 5-fold cross-validation (re-using all other parameters including the seed):

In [ ]:
gbm = h2o.get_model(sorted_final_grid['Model Id'][0])
params = gbm.params
new_params = {"nfolds":5, "model_id":None}
for key in new_params.keys():
    params[key]['actual'] = new_params[key]
gbm_best = h2o.H2OGradientBoostingEstimator()
for key in params.keys():
    if key in dir(gbm_best) and getattr(gbm_best,key) != params[key]['actual']:
        setattr(gbm_best,key,params[key]['actual']) 

In [ ]:
dir(gbm)

In [ ]:
gbm_best.train(x=predictors, y=response, training_frame=df)

In [ ]:
print gbm_best.cross_validation_metrics_summary()

Ouch! So it looks like we overfit quite a bit on the validation set as the mean AUC on the 5 folds is "only" 96.55% +/- 0.5%. So we cannot always expect AUCs of 97% with these parameters on this dataset. So to get a better estimate of model performance, the Random hyper-parameter search could have used `nfolds = 5` (or 10, or similar) in combination with 80% of the data for training (i.e., not holding out a validation set, but only the final test set). However, this would take more time, as `nfolds+1` models will be built for every set of parameters.

Instead, to save time, lets just scan through the top 5 models and cross validate their parameters with `nfolds = 5` on the entire dataset:

In [ ]:
for i in range(5):
    gbm = h2o.get_model(sorted_final_grid['Model Id'][i])
    params = gbm.params
    new_params = {"nfolds":5, "model_id":None}
    for key in new_params.keys():
        params[key]['actual'] = new_params[key]
    new_model = h2o.H2OGradientBoostingEstimator()
    for key in params.keys():
        if key in dir(new_model) and getattr(new_model,key) != params[key]['actual']:
            setattr(new_model,key,params[key]['actual'])
    new_model.train(x = predictors, y = response, training_frame = df)  
    cv_summary = new_model.cross_validation_metrics_summary().as_data_frame()
    print(gbm.model_id)
    print(cv_summary.iloc[4]) ## AUC

The avid reader might have noticed that we just implicitly did further parameter tuning using the "final" test set (which is part of the entire dataset `df`), which is not good practice - one is not supposed to use the "final" test set more than once. Hence, we're not going to pick a different "best" model, but we're just learning about the variance in AUCs. It turns out, for this tiny dataset, that the variance is rather large, which is not surprising.

Keeping the same "best" model, we can make test set predictions as follows:

In [ ]:
preds = best_model.predict(test)
preds.head()

Note that the label (survived or not) is predicted as well (in the first predict column), and it uses the threshold with the highest F1 score (here: 0.528098) to make labels from the probabilities for survival (`p1`). The probability for death (`p0`) is given for convenience, as it is just `1-p1`.

In [ ]:
best_model.model_performance()

You can also see the "best" model in more detail in [Flow](http://localhost:54321/):
![alt text](./best_gbm1.png "Best GBM from Grid Search in Flow")
![alt text](./best_gbm2.png "Best GBM from Grid Search in Flow")

The model and the predictions can be saved to file as follows:

In [ ]:
h2o.save_model(best_model, "/tmp/bestModel.csv", force=True)
h2o.export_file(preds, "/tmp/bestPreds.csv", force=True)

In [ ]:
#h2o.download_pojo(best_model)

The model can also be exported as a plain old Java object (POJO) for H2O-independent (standalone/Storm/Kafka/UDF) scoring in any Java environment.

```
/*
  Licensed under the Apache License, Version 2.0
    http://www.apache.org/licenses/LICENSE-2.0.html

  AUTOGENERATED BY H2O at 2016-05-27T22:11:18.485-07:00
  3.8.2.6
  
  Standalone prediction code with sample test data for GBMModel named final_grid_model_46

  How to download, compile and execute:
      mkdir tmpdir
      cd tmpdir
      curl http://127.0.0.1:54321/3/h2o-genmodel.jar > h2o-genmodel.jar
      curl http://127.0.0.1:54321/3/Models.java/final_grid_model_46 > final_grid_model_46.java
      javac -cp h2o-genmodel.jar -J-Xmx2g -J-XX:MaxPermSize=128m final_grid_model_46.java

     (Note:  Try java argument -XX:+PrintCompilation to show runtime JIT compiler behavior.)
*/
...
class final_grid_model_46_Tree_289_class_0 {
  static final double score0(double[] data) {
    double pred =      (!Double.isNaN(data[9]) && data[9 /* boat */] != 12.0f ? 
         (data[9 /* boat */] <13.003977f ? 
             (data[9 /* boat */] <11.504464f ? 
                 (data[1 /* sex */] <0.5f ? 
                     (data[6 /* fare */] <24.978184f ? 
                        0.0027945167f : 
                         (data[11 /* home.dest */] <132.0f ? 
                            0.0027875851f : 
                             (data[11 /* home.dest */] <265.5f ? 
                                 (data[6 /* fare */] <49.690052f ? 
                                    0.002784563f : 
                                    0.0027828787f) : 
                                0.0027856496f))) : 
                    0.002798641f) : 
                0.0028186426f) : 
            0.00350058f) : 
...
```

## Ensembling Techniques

After learning above that the variance of the test set AUC of the top few models was rather large, we might be able to turn this into our advantage by using ensembling techniques. The simplest one is taking the average of the predictions (survival probabilities) of the top `k` grid search model predictions (here, we use `k=10`):

In [ ]:
prob = None
k=10
for i in range(0,k): 
    gbm = h2o.get_model(sorted_final_grid["Model Id"][i])
    if (prob is None):
        prob = gbm.predict(test)["p1"]
    else:
        prob = prob + gbm.predict(test)["p1"]
prob = prob/k

We now have a blended probability of survival for each person on the Titanic.

In [ ]:
prob.head()

We can bring those ensemble predictions to our Python session's memory space and use other Python packages.

In [ ]:
from sklearn.metrics import roc_auc_score
# convert prob and test[response] h2oframes to pandas' frames and then convert them each to numpy array
np_array_prob = prob.as_data_frame().as_matrix()
np_array_test = test[response].as_data_frame().as_matrix()
probInPy = np_array_prob
labeInPy = np_array_test
# compare true scores (test[response]) to probability scores (prob)
roc_auc_score(labeInPy, probInPy)

This simple blended ensemble test set prediction has an even higher AUC than the best single model, but we need to do more validation studies, ideally using cross-validation. We leave this as an exercise for the reader - take the parameters of the top `10` models, retrain them with `nfolds=5` on the full dataset, set `keep_holdout_predictions=TRUE` and sum up their predicted probabilities, then score that with sklearn's roc_auc_score as shown above.

For more sophisticated ensembling approaches, such as stacking via a superlearner, we refer to the [H2O Ensemble](https://github.com/h2oai/h2o-3/tree/master/h2o-r/ensemble) github page.

## Summary
We learned how to build H2O GBM models for a binary classification task on a small but realistic dataset with numerical and categorical variables, with the goal to maximize the AUC (ranges from 0.5 to 1). We first established a baseline with the default model, then carefully tuned the remaining hyper-parameters without "too much" human guess-work. We used both Cartesian and Random hyper-parameter searches to find good models. We were able to get the AUC on a holdout test set from the low 94% range with the default model to the mid 97% after tuning, and to the high 97% with some simple ensembling technique known as blending. We performed simple cross-validation variance analysis to learn that results were slightly "lucky" due to the specific train/valid/test set splits, and settled to expect mid 96% AUCs instead.

Note that this script and the findings therein are directly transferrable to large datasets on distributed clusters including Spark/Hadoop environments.

More information can be found here [http://www.h2o.ai/docs/](http://www.h2o.ai/docs/).